In [12]:
# Import modules to make a connection to DB and query
import pandas as pd
from sqlalchemy import create_engine

In [37]:
# DB credentials
db_username = 'db_username'
db_password = 'db_password'
db_host = 'db_host'
db_port = 'db_port'
db_name = 'db_name'

In [14]:
# Create connection with DB credentials
engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

# CIS Queries

### Descriptive Query

#### Business Question: How does the average fuel efficiency for Tesla models compare to other brands by car class?

In [31]:
CIS_BQ_sql_query = """
WITH Efficiency AS (
  SELECT
    class,
    make,
    AVG(city_mpg) AS AvgCityMPG,
    AVG(highway_mpg) AS AvgHighwayMPG,
    AVG(combination_mpg) AS AvgCombinationMPG
  FROM ModelCarSpecs
  GROUP BY class, make
)
SELECT
  class,
  make,
  AVG(AvgCityMPG) AS OverallAvgCityMPG,
  AVG(AvgHighwayMPG) AS OverallAvgHighwayMPG,
  AVG(AvgCombinationMPG) AS OverallAvgCombinationMPG
FROM Efficiency
GROUP BY class, make
ORDER BY class, make;
"""

In [32]:
CIS_BQ_sql_results = pd.read_sql_query(CIS_BQ_sql_query, con=engine)

In [35]:
print(CIS_BQ_sql_results.head())

                         class     make  OverallAvgCityMPG  \
0                    large car  hyundai              123.0   
1                    large car    tesla               90.2   
2                  midsize car    tesla              127.0   
3  small sport utility vehicle  hyundai              129.5   
4  small sport utility vehicle    tesla              123.6   

   OverallAvgHighwayMPG  OverallAvgCombinationMPG  
0                  93.0                  107.3333  
1                  92.6                   91.2000  
2                 116.8                  122.2000  
3                  96.0                  112.0000  
4                 111.0                  117.8000  


In [34]:
CIS_BQ_sql_results.to_csv('CIS_BQ_sql_results.txt', sep='|', index=False)

### Insight:

#### Tesla has models in large Car Classes, has the only Midsize model, and offers small sport utility model. With no offerings in the small station wagon car class. The mgpe for Tesla models seems to lag behind other competitors in certain classes like Large cars. 

### Recommendation:

#### Revamp Teslas MPGe in large car class to keep up with Hyundais Ioniq 5, as there is a 10 mile gap in average MPGe. Possibly offer a model in the small stationwagon class to increase product offerings. Teslas small sport utility offering already has the highest average MPG compared to competitors.

### Prediction:

#### Increasing MPG on Tesla models classified as Large Cars would make Teslas model hold the highest average MPGe in the large car class, possibly being more attractive to consumers. Offering a model in the Small Stationwagon class would expand Teslas product offerings reach, applealing to more customers. 

### Diagnostic Query:

#### Business Question: What class & models yeild the highest average MPG?

In [27]:
CIS_DQ_sql_query = """
WITH Efficiency AS (
  SELECT
    MCS.class,
    MCS.make,
    MCS.model,
    MCS.ModelID,
    MCS.drive,
    MCS.transmission,
    MCS.fuel_type,
    AVG(MCS.combination_mpg) AS AvgCombinationMPG,
    DENSE_RANK() OVER (PARTITION BY MCS.class ORDER BY AVG(MCS.combination_mpg) DESC) AS EfficiencyRank
  FROM ModelCarSpecs MCS
  GROUP BY MCS.class, MCS.make, MCS.model, MCS.drive, MCS.transmission, MCS.fuel_type
),
CustomerSatisfaction AS (
  SELECT
    ModelID,
    AVG(CAST(overall_rating AS DECIMAL(10,2))) AS AvgSatisfaction -- Ensure that overall_rating is converted to a numeric type for averaging
  FROM CustomerReviews
  GROUP BY ModelID
)
SELECT
  E.class,
  E.make,
  E.model,
  E.drive,
  E.AvgCombinationMPG,
  E.EfficiencyRank,
  CS.AvgSatisfaction
FROM Efficiency E
LEFT JOIN CustomerSatisfaction CS ON E.ModelID = CS.ModelID
WHERE E.EfficiencyRank IN (1,2,3)
ORDER BY E.class, E.AvgCombinationMPG DESC;
"""

In [28]:
CIS_DQ_sql_results = pd.read_sql_query(CIS_DQ_sql_query, con=engine)

In [29]:
print(CIS_DQ_sql_results.head())

         class     make                         model drive  \
0    large car  hyundai      ioniq 5 rwd (long range)   rwd   
1    large car  hyundai  ioniq 5 rwd (standard range)   rwd   
2    large car  hyundai      ioniq 5 awd (long range)   awd   
3  midsize car    tesla            model 3 long range   rwd   
4  midsize car    tesla             model 3 mid range   rwd   

   AvgCombinationMPG  EfficiencyRank  AvgSatisfaction  
0              114.0               1         4.441176  
1              110.0               2         4.441176  
2               98.0               3         4.441176  
3              128.0               1         4.414141  
4              123.0               2         4.414141  


In [30]:
CIS_DQ_sql_results.to_csv('CIS_DQ_sql_results.txt', sep='|', index=False)

### Insight:

#### Teslas model S offering in the large car class did not make the top 3 ranking for most efficent per class, as Hyundais Ioniq 5 models have higher average MPGe. Some Tesla models hold the number 1 ranking in highest average MPGe like the Model 3 Long Range for Midsize car class and Model Y Long Range awd for Small Sport Utility Vehicle class. Models with fwd and rwd seem to have higher average MPGe. 

### Recommendation:

#### Increase MPGe in Tesla models in Large Car class to better compete with other manufacturers like Hyundai.

### Prediction:

#### By having a more competitive models with on par or higher average MPG, this would increase the attractiveness of Tesla's product offerings in Large Car class leading to more sales.

# NHTSA Queries

### Descriptive Query:

#### Business Question: What are the overall incident counts for each car make, highlighting potential differences in reliability or maintenance issues?

In [15]:
NHTSA_BQ_sql_query = """
WITH IssueCounts AS (
  SELECT
    ProductMake,
    ProductModel,
    Component,
    COUNT(*) AS IssueCount,
    RANK() OVER (PARTITION BY ProductModel ORDER BY COUNT(*) DESC) AS IssueRank
  FROM MaintenanceInfo
  GROUP BY ProductMake, ProductModel, Component
)
SELECT
  ProductMake,
  ProductModel,
  Component,
  IssueCount
FROM IssueCounts
WHERE IssueRank = 1;
"""

In [16]:
NHTSA_BQ_sql_results = pd.read_sql_query(NHTSA_BQ_sql_query, con=engine)

In [17]:
print(NHTSA_BQ_sql_results.head())

  ProductMake ProductModel                                   Component  \
0   CHEVROLET      BOLT EV                           EXTERIOR LIGHTING   
1  VOLKSWAGEN         ID.4                            UNKNOWN OR OTHER   
2     HYUNDAI      IONIQ 5                           ELECTRICAL SYSTEM   
3       TESLA      MODEL 3  FORWARD COLLISION AVOIDANCE,LANE DEPARTURE   
4       TESLA      MODEL 3                 FORWARD COLLISION AVOIDANCE   

   IssueCount  
0           4  
1          11  
2          77  
3           2  
4           2  


In [11]:
NHTSA_BQ_sql_results.to_csv('NHTSA_BQ_sql_results.txt', sep='|', index=False)

### Insight:

#### The most common component issues between all three Tesla models of this dataset were, for the Tesla Model 3 having a tie of two common issues being; Forward Collision Avoidance, Land departure and Forward Collision Avoidance. The Model S had the most common component issue of Electrical System, and the Model Y had the most common issue of Forward Collision Avoidance. 

### Recommendation:

#### Evaluate systems and related product engineering aspects related to Forward collision avoidance to better streamline the saftey of the model as it seems like a common issue for specifcaly the Model Y.

### Prediction:

#### By implementing redundancy and safety measures within specifically the Model Y, this would increase the attractiveness of models to customers.

### Diagnostic Query:

#### Business Question: How do maintenance issue frequencies for Tesla models compare to their competitors, and what are the specific common issues for each category of cars?

In [18]:
NHTSA_DQ_sql_query = """
SELECT
    IncidentYear,
    ProductModel,
    Component,
    UniqueIssueCount,
    AvgCityMPG,
    AvgHighwayMPG,
    IncidentRank
FROM (
    SELECT
        YEAR(mi.IncidentDate) AS IncidentYear,
        mym.BaseModel AS ProductModel,
        mi.Component,
        COUNT(DISTINCT mi.ODINumber) AS UniqueIssueCount,
        AVG(mcs.city_mpg) AS AvgCityMPG,
        AVG(mcs.highway_mpg) AS AvgHighwayMPG,
        ROW_NUMBER() OVER (
            PARTITION BY YEAR(mi.IncidentDate), mym.BaseModel
            ORDER BY COUNT(DISTINCT mi.ODINumber) DESC
        ) AS IncidentRank
    FROM 
        MaintenanceInfo mi
    INNER JOIN 
        ModelYearMapping mym ON mi.ProductModel = mym.BaseModel
    LEFT JOIN 
        ModelCarSpecs mcs ON mym.SpecificModel = mcs.model AND mym.ModelYear = mcs.Year
    GROUP BY 
        YEAR(mi.IncidentDate),
        mym.BaseModel,
        mi.Component
) RankedIncidents
WHERE 
    IncidentRank = 1
ORDER BY 
    IncidentYear DESC,
    UniqueIssueCount DESC;
"""

In [19]:
NHTSA_DQ_sql_results = pd.read_sql_query(NHTSA_DQ_sql_query, con=engine)

In [20]:
print(NHTSA_DQ_sql_results.head())

   IncidentYear ProductModel          Component  UniqueIssueCount  AvgCityMPG  \
0          2024      Ioniq 5  ELECTRICAL SYSTEM                 9       125.6   
1          2024      Model Y   UNKNOWN OR OTHER                 2       123.6   
2          2024         ID.4  ELECTRICAL SYSTEM                 1       103.0   
3          2024      Bolt EV   UNKNOWN OR OTHER                 1       127.6   
4          2023      Ioniq 5  ELECTRICAL SYSTEM                61       125.6   

   AvgHighwayMPG  IncidentRank  
0           94.2             1  
1          111.0             1  
2           89.4             1  
3          109.2             1  
4           94.2             1  


In [21]:
NHTSA_DQ_sql_results.to_csv('NHTSA_DQ_sql_results.txt', sep='|', index=False)

In [22]:
engine.dispose()

### Insight:

#### In the year 2022 the Model Y experienced a spike in issue complaints about Forward Collision Avoidance, showing a decrease in issues complaints in the next year 2023. Followed by no complaints in the year 2024. In the year 2020 the Model S experienced a spike in issue complaints relating to Electrical Systems Component with following years showing very little to none complaints related to Electrical systems. These issues compared to the number of issues from competitors are relatively below the upper bound.

### Recommendation:

#### Diagnose and source issues related to Forward Collision Avoidance, as it's a common issue between the Tesla model Y & 3. Despite marginal improvement in the decrease of issues related to Forward Collistion Avoidance in recent years.

### Prediction:

#### By refining product components, models will be viewed as a more attractive compared alternative competitor models experiencing more issues.